In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn import tree

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('../data/processed/all_rooms_combined.csv', index_col=0)

print(f'Наблюдений в датасете: {df.shape[0]}')

Наблюдений в датасете: 19144


In [3]:
df.head()

,rooms,subway,admin_okrug,district,street,home_number,price,year_of_construction,total_meters,kitchen_meters,flat_type,house_type,"dist_to_subway, min",way_to_subway,is_euro,is_skyscraper,floor_type,wc_count,wc_type,class_real
0,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Алиа ЖК,к7,16781328.0,2023.0,37.0,11.0,Новостройка,Монолитный,5.0,пешком,False,False,usual,1,совмещенный,премиум
1,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к7,16500000.0,2024.0,42.0,12.0,Новостройка,Монолитный,14.0,пешком,False,False,usual,1,совмещенный,бизнес
2,1,Стрешнево,СЗАО,р-н Щукино,Щукинская улица,7/9С7,20540352.0,2023.0,47.0,16.0,Новостройка,Монолитный,10.0,пешком,False,False,usual,2,совмещенный,премиум
3,1,Шелепиха,СЗАО,р-н Хорошево-Мневники,Шелепихинская набережная,34к3,20800000.0,2020.0,32.0,11.0,Вторичка,Монолитный,14.0,пешком,False,False,view,1,совмещенный,премиум
4,1,Спартак,СЗАО,р-н Покровское-Стрешнево,Северо-Западный ао,Клубный Город на Реке Примавера ЖК,30460120.0,2024.0,52.0,11.0,Новостройка,Монолитный,14.0,пешком,False,False,usual,1,раздельный,премиум


In [6]:
df.class_real.unique()

array(['премиум', 'бизнес', 'комфорт', 'эконом', 'элитный'], dtype=object)

In [4]:
df.dtypes

rooms                     int64
subway                   object
admin_okrug              object
district                 object
street                   object
home_number              object
price                   float64
year_of_construction    float64
total_meters            float64
kitchen_meters          float64
flat_type                object
house_type               object
dist_to_subway, min     float64
way_to_subway            object
is_euro                    bool
is_skyscraper              bool
floor_type               object
wc_count                  int64
wc_type                  object
class_real               object
dtype: object

In [5]:
log = ['kitchen_meters', 'total_meters']
categorical = ['way_to_subway', 'subway', 'floor_type', 'wc_type', 'admin_okrug', 'is_skyscraper', 'house_type', 'flat_type','district', 'class_real',]
num = ['dist_to_subway, min', 'wc_count']
ordinal = ['rooms', 'year_of_construction']
boolean = ['is_euro', 'is_skyscraper']

### Разбиваем на X и y

In [6]:
log = ['total_meters', 'kitchen_meters']
categorical = ['admin_okrug', 'subway', 'is_skyscraper', 'class_real', 'way_to_subway']
ordinal = ['rooms']

In [7]:
X = df[log + categorical + ordinal]
y = df['price']/df['total_meters']

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [10]:
X_train.head(2)

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms
15780,21.0,5.0,СВАО,Отрадное,False,бизнес,на транспорте,9
1177,33.0,15.0,НАО (Новомосковский),Филатов Луг,False,комфорт,пешком,1


In [11]:
def log_transform(x):
    print(x)
    return np.log(x + 1)

In [12]:
log_transformer = FunctionTransformer(log_transform)

In [13]:
col_transformer = ColumnTransformer([("Log transform", log_transformer, log),
                                ("Scale", StandardScaler(), ordinal),
                                ("One hot", OneHotEncoder(sparse=False, handle_unknown='ignore'),categorical)],
                                remainder="passthrough")
X_train_transformed = col_transformer.fit_transform(X_train)

       total_meters  kitchen_meters
15780          21.0             5.0
1177           33.0            15.0
1112           43.0            14.0
941            58.0            10.0
8980           71.0            28.0
...             ...             ...
17960          29.0            10.0
6932           65.0            12.0
13996         147.0            18.0
237            35.0            11.0
15173         201.0            16.0

[12966 rows x 2 columns]


In [14]:
X_test_transformed = col_transformer.transform(X_test)

       total_meters  kitchen_meters
16769           7.0             2.0
2532           37.0            10.0
11934         169.0            25.0
7643           42.0            10.0
17832          21.0             5.0
...             ...             ...
17326          24.0             2.0
2939           24.0            10.0
12999          94.0            22.0
6745           58.0            21.0
5552           68.0            20.0

[3242 rows x 2 columns]


In [15]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import catboost as ctb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [16]:
df_test = pd.merge(X_test, pd.DataFrame(y_test, columns=['real']), how = 'left', left_index=True, right_index=True)

### Простая линейная регрессия ###

In [168]:
linreg = LinearRegression().fit(X_train_transformed, y_train)
MAE_lr = round(mean_absolute_error(y_test, linreg.predict(X_test_transformed)), 3)
MAE_lr

4510970674390.974

In [169]:
df_test['preds'] = linreg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","308,440.00"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","376,688.00"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,030,120.00"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","469,888.00"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","469,576.00"


In [173]:
df_test['diff'] = df_test['real']-df_test['preds']

In [174]:
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
9886,89.00,12.00,ЦАО,Добрынинская,False,премиум,пешком,3,"421,348.31","520,058,304,129,296.00","-520,058,303,707,947.69"
17197,18.00,3.00,СВАО,Лось,False,комфорт,на транспорте,9,"272,222.22","520,058,303,865,032.00","-520,058,303,592,809.75"
9662,50.00,7.00,СВАО,Лось,False,эконом,на транспорте,3,"230,000.00","520,058,303,816,536.00","-520,058,303,586,536.00"
3048,39.00,9.00,ЮЗАО,Битца,False,комфорт,на транспорте,1,"266,666.67","520,058,303,816,520.00","-520,058,303,549,853.31"
1910,43.00,10.00,ЮАО,Бирюлёво-Пассажирская,False,комфорт,пешком,1,"259,302.33","520,058,303,808,936.00","-520,058,303,549,633.69"
...,...,...,...,...,...,...,...,...,...,...,...
9604,202.00,12.00,ЦАО,Парк Культуры,False,элитный,пешком,3,"2,289,287.87","1,224,552.00","1,064,735.87"
10444,120.00,12.00,ЦАО,Лубянка,False,элитный,пешком,3,"2,337,411.54","1,060,088.00","1,277,323.54"
13558,302.00,10.00,ЦАО,Пушкинская,False,элитный,пешком,5,"2,423,663.58","1,112,264.00","1,311,399.58"
11954,459.00,30.00,ЦАО,Третьяковская,False,элитный,пешком,5,"4,476,318.08","1,738,440.00","2,737,878.08"


### Ridge-регрессия ###

In [176]:
ridge = Ridge(alpha=1000).fit(X_train_transformed, y_train)
MAE_r = round(mean_absolute_error(y_test, ridge.predict(X_test_transformed)), 3)
MAE_r

74763.86

In [177]:
df_test['preds'] = ridge.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","304,507.11","12,988.57"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","367,926.27","-49,660.97"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","926,667.24","172,010.18"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","583,372.46","-27,050.86"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","420,030.39","208,369.52"


In [178]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
14116,388.00,30.00,ЦАО,Шаболовская,False,комфорт,пешком,5,"283,247.42","589,839.51","-306,592.09"
12084,225.00,33.00,ЦАО,Октябрьская,False,бизнес,пешком,5,"311,111.11","609,548.70","-298,437.59"
14385,260.00,20.00,ЦАО,Шелепиха,False,комфорт,пешком,5,"288,461.54","566,721.32","-278,259.78"
13581,190.00,20.00,ЦАО,Чистые пруды,False,премиум,пешком,5,"421,052.63","694,409.26","-273,356.63"
14909,224.00,25.00,ЦАО,Тверская,False,премиум,пешком,5,"436,160.71","700,756.57","-264,595.85"
...,...,...,...,...,...,...,...,...,...,...,...
4571,234.00,60.00,ЦАО,Киевская,False,элитный,на транспорте,2,"2,390,384.62","951,428.15","1,438,956.47"
10461,82.00,6.00,ЦАО,Киевская,False,элитный,пешком,3,"2,344,408.59","892,161.67","1,452,246.92"
10444,120.00,12.00,ЦАО,Лубянка,False,элитный,пешком,3,"2,337,411.54","882,613.66","1,454,797.88"
13558,302.00,10.00,ЦАО,Пушкинская,False,элитный,пешком,5,"2,423,663.58","933,470.65","1,490,192.93"


### Lasso регрессия ###

In [179]:
lasso_reg = Lasso(alpha = 2.65e-05).fit(X_train_transformed, y_train)
MAE_lasso = round(mean_absolute_error(y_test, lasso_reg.predict(X_test_transformed)), 3)
MAE_lasso

/Users/max/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.824e+13, tolerance: 9.618e+10
  model = cd_fast.enet_coordinate_descent(


51763.021

In [180]:
df_test['preds'] = lasso_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","308,156.38","16,921.46"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","376,440.05","-40,899.24"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,030,115.95","275,462.94"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","469,820.13","-140,535.32"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","469,348.33","257,915.14"


In [181]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
12215,210.00,20.00,ЦАО,Третьяковская,False,элитный,пешком,5,"952,380.95","1,726,073.73","-773,692.78"
14907,103.00,13.00,ЦАО,Киевская,False,элитный,пешком,5,"718,446.60","1,453,890.12","-735,443.52"
11676,186.00,17.00,ЦАО,Третьяковская,False,премиум,пешком,5,"586,005.38","1,310,369.83","-724,364.45"
13581,190.00,20.00,ЦАО,Чистые пруды,False,премиум,пешком,5,"421,052.63","1,107,672.43","-686,619.80"
14299,164.00,18.00,ЗАО,Киевская,False,элитный,на транспорте,5,"701,219.51","1,324,256.05","-623,036.53"
...,...,...,...,...,...,...,...,...,...,...,...
4571,234.00,60.00,ЦАО,Киевская,False,элитный,на транспорте,2,"2,390,384.62","1,479,645.08","910,739.54"
9604,202.00,12.00,ЦАО,Парк Культуры,False,элитный,пешком,3,"2,289,287.87","1,224,596.32","1,064,691.55"
10444,120.00,12.00,ЦАО,Лубянка,False,элитный,пешком,3,"2,337,411.54","1,060,104.98","1,277,306.56"
13558,302.00,10.00,ЦАО,Пушкинская,False,элитный,пешком,5,"2,423,663.58","1,112,358.47","1,311,305.10"


### Decision Tree ###

In [185]:
dt_reg = DecisionTreeRegressor(random_state = 0, max_depth=15).fit(X_train_transformed, y_train)
MAE_dt = round(mean_absolute_error(y_test, dt_reg.predict(X_test_transformed)), 3)
MAE_dt

41546.134

In [186]:
df_test['preds'] = dt_reg.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","340,428.81"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","340,428.81"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,039,768.76"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","466,528.78"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","474,703.20"


In [187]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
15170,125.00,7.00,ЦАО,Тверская,False,элитный,пешком,5,"768,000.00","3,000,944.88","-2,232,944.88"
12668,419.00,18.00,ЗАО,Ломоносовский проспект,False,элитный,на транспорте,5,"921,412.89","1,999,632.55","-1,078,219.66"
11206,105.00,10.00,ЗАО,Ломоносовский проспект,False,элитный,пешком,3,"700,000.00","1,545,000.00","-845,000.00"
12495,206.00,25.00,ЦАО,Киевская,False,элитный,на транспорте,5,"1,224,455.83","1,971,323.53","-746,867.70"
11673,239.00,82.00,ЦАО,Спортивная,False,элитный,пешком,5,"1,614,965.25","2,265,392.47","-650,427.22"
...,...,...,...,...,...,...,...,...,...,...,...
9604,202.00,12.00,ЦАО,Парк Культуры,False,элитный,пешком,3,"2,289,287.87","1,039,768.76","1,249,519.11"
10444,120.00,12.00,ЦАО,Лубянка,False,элитный,пешком,3,"2,337,411.54","936,300.46","1,401,111.08"
10461,82.00,6.00,ЦАО,Киевская,False,элитный,пешком,3,"2,344,408.59","900,139.24","1,444,269.34"
13558,302.00,10.00,ЦАО,Пушкинская,False,элитный,пешком,5,"2,423,663.58","940,484.69","1,483,178.89"


**Среди базовых моделей лучший результат без настройки гиперпараметров по метрике MAE показывают решающие деревья. Проведем настройку гиперпараметров по кросс-валидации**

In [189]:
from sklearn.model_selection import GridSearchCV

In [190]:
criterion = ['squared_error', 'absolute_error']
splitter = ['best', 'random']
max_depth = [5, 10, 20, 30, 40, 50]

In [191]:
grid = [{'criterion':criterion,
        'splitter':splitter,
        'max_depth':max_depth}]

gs = GridSearchCV(estimator=DecisionTreeRegressor(),
                  param_grid = grid,
                  scoring='neg_mean_absolute_error',
                  cv = 10,
                  n_jobs = -1)

In [192]:
gs = gs.fit(X_train_transformed, y_train)

In [193]:
-1*gs.best_score_

42125.03892028832

In [194]:
gs.best_params_

{'criterion': 'absolute_error', 'max_depth': 40, 'splitter': 'random'}

In [195]:
gs.score(X_test_transformed, y_test)

-38589.45594222635

In [196]:
gs.best_estimator_

DecisionTreeRegressor(criterion='absolute_error', max_depth=40,
                      splitter='random')

In [197]:
df_test['preds'] = gs.best_estimator_.predict(X_test_transformed)
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","292,307.69","-19,000.24"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","335,000.00","-13,401.79"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,103,905.33","162,361.42"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","466,528.78","-23,691.64"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","447,761.19","203,242.32"


In [198]:
df_test['diff'] = df_test['real']-df_test['preds']
df_test.sort_values(by='diff')

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff
7386,119.00,31.00,ЦАО,Киевская,False,элитный,на транспорте,2,"966,386.55","2,550,487.80","-1,584,101.25"
12495,206.00,25.00,ЦАО,Киевская,False,элитный,на транспорте,5,"1,224,455.83","2,477,672.81","-1,253,216.99"
9060,150.00,15.00,ЦАО,Третьяковская,False,элитный,пешком,3,"1,193,333.33","2,356,765.15","-1,163,431.82"
12668,419.00,18.00,ЗАО,Ломоносовский проспект,False,элитный,на транспорте,5,"921,412.89","1,999,632.55","-1,078,219.66"
16443,232.00,2.00,ЮАО,Технопарк,False,элитный,пешком,9,"862,197.17","1,888,412.02","-1,026,214.84"
...,...,...,...,...,...,...,...,...,...,...,...
11679,433.00,37.00,ЦАО,Улица 1905 года,False,элитный,пешком,5,"1,653,391.45","720,000.00","933,391.45"
10459,110.00,4.00,ЦАО,Киевская,False,элитный,пешком,3,"2,329,340.05","1,106,557.38","1,222,782.67"
10461,82.00,6.00,ЦАО,Киевская,False,элитный,пешком,3,"2,344,408.59","1,106,557.38","1,237,851.21"
10444,120.00,12.00,ЦАО,Лубянка,False,элитный,пешком,3,"2,337,411.54","886,363.64","1,451,047.91"


In [199]:
df_test['price_real'] = df_test['total_meters'] * df_test['real']
df_test['price_predicted'] = df_test['total_meters'] * df_test['preds']
df_test['diff'] = df_test['price_real'] - df_test['price_predicted']

In [200]:
pd.options.display.float_format = '{:,.2f}'.format

In [201]:
df_test

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","292,307.69","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","335,000.00","-295,000.00","12,100,000.00","12,395,000.00"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,103,905.33","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","466,528.78","-995,048.78","18,599,160.00","19,594,208.78"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","447,761.19","4,833,870.93","14,236,856.00","9,402,985.07"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17326,24.00,2.00,ЗАО,Строгино,False,бизнес,пешком,9,"337,585.00","349,922.02","-296,088.46","8,102,040.00","8,398,128.46"
2939,24.00,10.00,СВАО,Алексеевская,False,премиум,пешком,1,"528,937.50","514,534.66","345,668.25","12,694,500.00","12,348,831.75"
12999,94.00,22.00,НАО (Новомосковский),Коммунарка,False,эконом,пешком,5,"196,808.51","196,808.51",0.00,"18,500,000.00","18,500,000.00"
6745,58.00,21.00,СЗАО,Пятницкое шоссе,False,эконом,на транспорте,2,"213,516.19","214,177.47","-38,354.00","12,383,939.00","12,422,293.00"


### CatBoost ###

In [354]:
X_train_ctb = X_train.copy()
X_test_ctb = X_test.copy()

In [355]:
X_train_ctb.dtypes

total_meters      float64
kitchen_meters    float64
admin_okrug        object
subway             object
is_skyscraper        bool
class_real         object
way_to_subway      object
rooms               int64
dtype: object

In [357]:
X_test_ctb.dtypes

total_meters      float64
kitchen_meters    float64
admin_okrug        object
subway             object
is_skyscraper        bool
class_real         object
way_to_subway      object
rooms               int64
dtype: object

In [358]:
X_train_ctb.drop('is_skyscraper', axis=1, inplace=True)
X_test_ctb.drop('is_skyscraper', axis=1, inplace=True)


In [359]:
X_train_ctb

,total_meters,kitchen_meters,admin_okrug,subway,class_real,way_to_subway,rooms
15780,21.00,5.00,СВАО,Отрадное,бизнес,на транспорте,9
1177,33.00,15.00,НАО (Новомосковский),Филатов Луг,комфорт,пешком,1
1112,43.00,14.00,ЮЗАО,Калужская,бизнес,пешком,1
941,58.00,10.00,ЦАО,Павелецкая,премиум,пешком,1
8980,71.00,28.00,ЗАО,Фили,премиум,пешком,3
...,...,...,...,...,...,...,...
17960,29.00,10.00,ЦАО,Красносельская,премиум,пешком,9
6932,65.00,12.00,ЮВАО,Некрасовка,эконом,пешком,2
13996,147.00,18.00,ЮЗАО,Нагорная,премиум,пешком,5
237,35.00,11.00,ЮЗАО,Ясенево,бизнес,на транспорте,1


In [360]:
X_train_ctb.dtypes == 'O'

total_meters      False
kitchen_meters    False
admin_okrug        True
subway             True
class_real         True
way_to_subway      True
rooms             False
dtype: bool

In [361]:
ctbst = ctb.CatBoostRegressor(cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                                 random_state = 42)
ctbst.fit(X_train_ctb, y_train, verbose = False)
#preds = ctbst.predict(X_test_ctb)

In [362]:
X_test_ctb[X_test_ctb.way_to_subway.isna()].index

Int64Index([5516], dtype='int64')

In [364]:
X_test_ctb.loc[X_test_ctb[X_test_ctb.way_to_subway.isna()].index, 'way_to_subway'] = 'пешком'

In [365]:
(X_test_ctb['total_meters']=='пешком').sum()

0

In [366]:
X_test_ctb.dtypes

total_meters      float64
kitchen_meters    float64
admin_okrug        object
subway             object
class_real         object
way_to_subway      object
rooms               int64
dtype: object

In [367]:
ctbst_preds = ctbst.predict(X_test_ctb)

In [368]:
len(ctbst_preds)

3242

In [370]:
df_test['preds'] = ctbst_preds
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","335,580.69","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","357,228.16","-295,000.00","12,100,000.00","12,395,000.00"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,085,436.03","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","466,360.82","-995,048.78","18,599,160.00","19,594,208.78"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","482,958.07","4,833,870.93","14,236,856.00","9,402,985.07"


In [371]:
MAE_ctbst = round(mean_absolute_error(y_test, ctbst_preds), 3)
MAE_ctbst

43013.282

In [379]:
ctbst = ctb.CatBoostRegressor(iterations = 1000, 
                             depth = 15, 
                             learning_rate = 0.04746000096201897,
                             random_strength = 1,
                             border_count = 254,
                             l2_leaf_reg = 3, 
                             grow_policy = 'SymmetricTree',
                             cat_features = list(X_train_ctb.columns[X_train_ctb.dtypes == 'O']),
                             random_state = 42)
ctbst.fit(X_train_ctb, y_train, verbose = False)

In [380]:
ctbst_preds = ctbst.predict(X_test_ctb)
MAE_ctbst = round(mean_absolute_error(y_test, ctbst_preds), 3)
MAE_ctbst

37538.141

In [381]:
df_test['preds'] = ctbst_preds
df_test.head()

,total_meters,kitchen_meters,admin_okrug,subway,is_skyscraper,class_real,way_to_subway,rooms,real,preds,diff,price_real,price_predicted
16769,7.00,2.00,ЗАО,Солнцево,False,бизнес,пешком,9,"321,428.57","328,386.85","203,846.15","2,250,000.00","2,046,153.85"
2532,37.00,10.00,ЗАО,Раменки,False,бизнес,пешком,1,"327,027.03","358,067.89","-295,000.00","12,100,000.00","12,395,000.00"
11934,169.00,25.00,ЦАО,Выставочная,True,элитный,пешком,5,"1,202,130.18","1,163,386.42","16,600,000.00","203,160,000.00","186,560,000.00"
7643,42.00,10.00,ЦАО,Электрозаводская,False,премиум,пешком,2,"442,837.14","435,793.31","-995,048.78","18,599,160.00","19,594,208.78"
17832,21.00,5.00,ЮАО,ЗИЛ,False,премиум,пешком,9,"677,945.52","492,418.53","4,833,870.93","14,236,856.00","9,402,985.07"


### RandomForest Regressor ###

In [17]:
rf = RandomForestRegressor(criterion='squared_error', n_estimators = 300)

In [18]:
rf.fit(X_train_transformed, y_train)

RandomForestRegressor(n_estimators=300)

In [19]:
rf_preds = rf.predict(X_test_transformed)
MAE_rf = round(mean_absolute_error(y_test, rf_preds), 3)
MAE_rf

32505.754

In [ ]:
rf2 = RandomForestRegressor(criterion='absolute_error', n_estimators = 100)
rf2.fit(X_train_transformed, y_train)

In [ ]:
rf2_preds = rf2.predict(X_test_transformed)
MAE_rf2 = round(mean_absolute_error(y_test, rf2_preds), 3)
MAE_rf2

In [203]:
# ### Гиперпараметры леса
# choices = pd.DataFrame(index = range(0, 1000, 1), columns = ['n_estimators',
#                                                             'max_depth',
#                                                             'min_samples_split',
#                                                             'min_samples_leaf',
#                                                             'max_features',
#                                                             'max_leaf_nodes',
#                                                             'bootstrap',
#                                                             'max_samples',
#                                                             'threshold'])
# # 'special' for None
# choices['n_estimators'] = np.random.RandomState(42).choice([100, 300, 500, 700, 1000]*200, 1000, replace = False)
# choices['max_depth'] = np.random.RandomState(43).choice([2, 4, 6, 8, 10]*200, 1000, replace = False)
# choices['min_samples_split'] = np.random.RandomState(44).choice([2, 5, 10, 20, 40]*200, 1000, replace = False)
# choices['min_samples_leaf'] = np.random.RandomState(45).choice([1, 2, 5, 10, 20]*200, 1000, replace = False)
# choices['max_features'] = np.random.RandomState(46).choice([0.7, 'log2', 'sqrt', 'auto']*250, 1000, replace = False)
# choices['max_leaf_nodes'] = np.random.RandomState(47).choice([5, 10, 20, 'special']*250, 1000, replace = False)
# choices['bootstrap'] = np.random.RandomState(48).choice([True]*800 + [False]*200, 1000, replace = False)
# choices['max_samples'] = np.random.RandomState(49).choice([0.5, 0.7, 0.9, 'special']*250, 1000, replace = False)
# choices['threshold'] = np.random.RandomState(50).choice([0.05, 0.01, 0.005, 0.001]*250, 1000, replace = False)

# choices.loc[~choices.max_features.str.contains('log2|sqrt|auto'), 'max_features'] = \
# choices.loc[~choices.max_features.str.contains('log2|sqrt|auto'), 'max_features'].astype(np.float64)
# choices.loc[choices.max_leaf_nodes != 'special', 'max_leaf_nodes'] = \
# choices.loc[choices.max_leaf_nodes != 'special', 'max_leaf_nodes'].astype(np.int64)
# choices.loc[choices.max_samples != 'special', 'max_samples'] = \
# choices.loc[choices.max_samples != 'special', 'max_samples'].astype(np.float64)

# ### Гиперпараметры бустинга
# choices_bstr = pd.DataFrame(index = range(0, 500, 1), columns = ['iterations', 
#                                                                  'depth',  
#                                                                  'learning_rate', 
#                                                                  'random_strength',  
#                                                                  'bagging_temperature', 
#                                                                  'border_count',  
#                                                                  'l2_leaf_reg',  
#                                                                  'grow_policy',  
#                                                                  'threshold'])
# choices_bstr['iterations'] = np.random.RandomState(42).choice([250, 500, 1000, 1500]*125, 500, replace = False)
# choices_bstr['depth'] = np.random.RandomState(43).choice([2, 4, 6, 8]*125, 500, replace = False)
# choices_bstr['learning_rate'] = np.random.RandomState(44).choice([0.001, 0.01, 0.1, 0.2, 0.3]*100, 500, replace = False)
# choices_bstr['random_strength'] = np.random.RandomState(45).choice([0, 0.3, 0.6, 1]*125, 500, replace = False)
# choices_bstr['bagging_temperature'] = np.random.RandomState(46).choice([0, 0.7, 0.9, 1]*125, 500, replace = False)
# choices_bstr['border_count'] = np.random.RandomState(47).choice([60, 128, 200, 254]*125, 500, replace = False)
# choices_bstr['l2_leaf_reg'] = np.random.RandomState(48).choice([1, 3, 5, 10]*125, 500, replace = False)
# choices_bstr['grow_policy'] = np.random.RandomState(49).choice(['SymmetricTree', 'Depthwise']*250, 500, replace = False)
# choices_bstr['threshold'] = np.random.RandomState(50).choice([0.01, 0.005, 0.001, 0.0005]*125, 500, replace = False)

**Можно сказать, что наибольшей важностью для правильности предсказания уровня арендной платы являются:**
- площадь квартиры
- административный район Москвы (расположение в ЦАО существеннее всего влияет на арендную плату)
- этаж и количество этажей в доме
- наличие посудомоечной машины